# Chapter 6 Notes 

# The Encoder

Looking to draw a line of connection between the Game State object created in chapter 3 and the encode. 
To visualize this concept we will use a super simple game state of a 3x3 tic-tac-toe board.

In [15]:
import numpy as np


class GameState:
    def __init__(self):
        self.board = [[' ' for _ in range(3)] for _ in range(3)]
    
    def place_stone(self, x, y, player):
        """Place a stone ('X' or 'O') at position (x, y)"""
        self.board[x][y] = player
    

# initializing the game state and making moves 
game_state = GameState()
game_state.place_stone(0, 0, 'X')
game_state.place_stone(1, 1, 'O')
game_state.place_stone(2, 2, 'X')

print("The board looks like in its 2D array (or a rank 2 tensor) form:", game_state.board )




The board looks like in its 2D array (or a rank 2 tensor) form: [['X', ' ', ' '], [' ', 'O', ' '], [' ', ' ', 'X']]


In [16]:
class SimpleEncoder:

    """
    This is the exact same encoder as the one in chapter 6 but scaled down to tic-tac-toe
    This is only an Index  encorder, not a value encorder (i.e. translated (row number,  column number) to some index in a list)
    """

    def name(self):
        return "SimpleEncoder"
    
    def encode(self, game_state):
        """
        Encode the game state into a flat list
        We are itterating over each 3 item row and putting each cell into a flat list 
        """

        flat_list = []
        for row in game_state.board:
            for cell in row:
                if cell == ' ':
                    flat_list.append(0)
                elif cell == 'X':
                    flat_list.append(1)
                elif cell == 'O':
                    flat_list.append(2)
        return np.array(flat_list)
    
    def encode_point(self, x, y):
        """
        Encode a point into an integer index
        
        Note: we calcuate the index only, the 
        """
        return 3 * x + y
    
    def decode_point_index(self, index):
        """Decoding an integer index into a point (x, y)"""
        return divmod(index, 3)
    
    # def num_points(self):
    #     return 3 * 3
    
    # def shape(self):
    #     return (3, 3)

encoder = SimpleEncoder()
encoded_state = encoder.encode(game_state)
print("Encoded State:", encoded_state)

encoded_point = encoder.encode_point(0, 1)
decoded_point = encoder.decode_point_index(encoded_point)
print(f"Encoded Point (0, 1) as index: {encoded_point}")
print(f"Decoded index {encoded_point} back to point: {decoded_point}")

Encoded State: [1 0 0 0 2 0 0 0 1]
Encoded Point (0, 1) as index: 1
Decoded index 1 back to point: (0, 1)


# Get Encoder by Type

Stating the obvious perhaps but this is useful because it will let us handle multiple encoders 

In [30]:
import importlib
import numpy as np
import enum
from collections import namedtuple

def get_encoder_by_name(name, board_size):
    if isinstance(board_size, int):
        board_size = (board_size, board_size)
    module = importlib.import_module('dlgo.encoders.' + name)
    constructor = getattr(module, 'create')
    return constructor(board_size)



In [31]:
# you interact with the unique encoders like so...

encoder1 = get_encoder_by_name('oneplane', 19)
encoder2 = get_encoder_by_name('simple', 19)

## Generate Go Game Data

In [ ]:
%%bash 

# this will generate 20 go games using Monte Calro as desribed in the book
python generate_mcts_games.py -n 20 --board-out features.npy --move-out labels.npy

# Keras API Overview 

In [6]:
%%bash 

export TF_CPP_MIN_LOG_LEVEL=3

In [ ]:
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # 0 is most logs, 3 is none

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense

In [3]:
# running a familiar example from chapter 5 but using Keras 


(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model = Sequential()
model.add(Dense(392, activation='sigmoid', input_shape=(784,)))
model.add(Dense(196, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 392)               307720    
                                                                 
 dense_1 (Dense)             (None, 196)               77028     
                                                                 
 dense_2 (Dense)             (None, 10)                1970      
                                                                 
Total params: 386718 (1.48 MB)
Trainable params: 386718 (1.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])

In [10]:
model.fit(x_train, y_train,
          batch_size=50,
          epochs=2)
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/2
1200/1200 [==============================] - 2s 1ms/step - loss: 0.0892 - accuracy: 0.2339
Epoch 2/2
313/313 [==============================] - 0s 944us/step - loss: 0.0891 - accuracy: 0.2535
Test loss: 0.08906501531600952
Test accuracy: 0.2535000145435333


# Back to Go bot training 

## Loading preprocessed data 

In [7]:
import numpy as np
import tensorflow as tf

np.random.seed(123)
X = np.load('/home/locutus/Desktop/dlgo_fork/deep_learning_and_the_game_of_go_with_notes/code/generated_games/features-20.npy')
Y = np.load('/home/locutus/Desktop/dlgo_fork/deep_learning_and_the_game_of_go_with_notes/code/generated_games/labels-20.npy')
samples = X.shape[0]
board_size = 9 * 9

X = X.reshape(samples, board_size)
Y = Y.reshape(samples, board_size)

train_samples = int(0.9 * samples)
X_train, X_test = X[:train_samples], X[train_samples:]
Y_train, Y_test = Y[:train_samples], Y[train_samples:]

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_shape=(board_size,)),
    tf.keras.layers.Dense(units=board_size, activation='softmax')
])

In [19]:
model = Sequential()
model.add(Dense(1000, activation='sigmoid', input_shape=(board_size,)))
model.add(Dense(500, activation='sigmoid'))
model.add(Dense(board_size, activation='sigmoid'))
model.summary()

model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(X_train, Y_train,
          batch_size=64,
          epochs=2,
          verbose=1,
          validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
 

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 1000)              82000     
                                                                 
 dense_6 (Dense)             (None, 500)               500500    
                                                                 
 dense_7 (Dense)             (None, 81)                40581     
                                                                 
Total params: 623081 (2.38 MB)
Trainable params: 623081 (2.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/2
18/18 [==============================] - 0s 8ms/step - loss: 0.2610 - accuracy: 0.0147 - val_loss: 0.2515 - val_accuracy: 0.0082
Epoch 2/2
18/18 [==============================] - 0s 3ms/step - loss: 0.2439 - accuracy: 0.0147 - val_loss: 0.2349 - val_accuracy: 0.0082
Test

In [20]:
test_board = np.array([[
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 1, -1,  1, -1, 0, 0, 0, 0,
    0, 0,  1, -1,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
    0, 0,  0,  0,  0, 0, 0, 0, 0,
]])
move_probs = model.predict(test_board)[0]
i = 0
for row in range(9):
    row_formatted = []
    for col in range(9):
        row_formatted.append('{:.3f}'.format(move_probs[i]))
        i += 1
    print(' '.join(row_formatted))

1/1 [==============================] - 0s 48ms/step
0.478 0.320 0.387 0.414 0.538 0.429 0.419 0.457 0.489
0.476 0.478 0.671 0.314 0.579 0.619 0.525 0.271 0.239
0.747 0.380 0.377 0.721 0.462 0.463 0.423 0.827 0.383
0.578 0.616 0.175 0.392 0.386 0.505 0.317 0.514 0.603
0.478 0.284 0.549 0.481 0.571 0.339 0.370 0.143 0.250
0.208 0.377 0.609 0.716 0.530 0.298 0.370 0.290 0.583
0.234 0.441 0.669 0.479 0.477 0.420 0.478 0.491 0.767
0.472 0.612 0.667 0.429 0.662 0.297 0.502 0.356 0.312
0.499 0.500 0.487 0.468 0.702 0.434 0.441 0.410 0.361


# Convolutional Networks 

In [9]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

np.random.seed(123)
X = np.load('/home/locutus/Desktop/dlgo_fork/deep_learning_and_the_game_of_go_with_notes/code/generated_games/features-20.npy')
Y = np.load('/home/locutus/Desktop/dlgo_fork/deep_learning_and_the_game_of_go_with_notes/code/generated_games/labels-20.npy')

samples = X.shape[0]
size = 9
input_shape = (size, size, 1)

X = X.reshape(samples, size, size, 1)

train_samples = int(0.9 * samples)
X_train, X_test = X[:train_samples], X[train_samples:]
Y_train, Y_test = Y[:train_samples], Y[train_samples:]


In [10]:
model = Sequential()
model.add(Conv2D(48, kernel_size=(3, 3),
                 activation='relu',
                 padding='same',
                 input_shape=input_shape))
model.add(Dropout(rate=0.5))
model.add(Conv2D(48, (3, 3),
                 padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(size * size, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 9, 9, 48)          480       
                                                                 
 dropout (Dropout)           (None, 9, 9, 48)          0         
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 48)          20784     
                                                                 
 max_pooling2d (MaxPooling2  (None, 4, 4, 48)          0         
 D)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 4, 4, 48)          0         
                                                                 
 flatten (Flatten)           (None, 768)               0         
                                                      

_________________________________________________________________


In [3]:
# # uncomment with causion, if your GPU is not powerful enough, this will crash your system

# model.fit(X_train, Y_train,
#           batch_size=64,
#           epochs=1,
#           verbose=1,
#           validation_data=(X_test, Y_test))
# score = model.evaluate(X_test, Y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])